Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.


# LAB9

Write a local-search algorithm (eg. an EA) able to solve the _Problem_ instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

- Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
- Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

- Reviews will be assigned on Monday, December 4
- You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)


In [129]:
# import random
# from tqdm import tqdm
# import lab9_lib

In [130]:
# fitness = lab9_lib.make_problem(1)
# for n in range(10):
#     ind = random.choices([0, 1], k=10)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

# print(fitness.calls)

The Python code in the lib defines an abstract class `AbstractProblem` and a function `make_problem(a)`.

The `AbstractProblem` class has the following methods and properties:

- `__init__`: Initializes the instance variable `_calls` to 0.
- `x`: An abstract property. Subclasses of `AbstractProblem` are expected to implement this.
- `calls`: A property that returns the number of times the instance has been called.
- `onemax`: A static method that takes a genome (a sequence of genes) and returns the sum of its genes, treating each gene as a boolean value.
- `__call__`: A special method that allows instances of the class to be called like functions. It increments `_calls`, computes fitnesses of the genome by slicing it into segments of length `x` and applying `onemax` to each segment, and then computes a value based on these fitnesses.

The `make_problem(a)` function defines a subclass of `AbstractProblem` with `x` implemented as a property that always returns `a`, and returns this subclass. This allows you to create problems with different values of `x` easily.

In summary, this code provides a framework for defining and working with optimization problems where the goal is to maximize the sum of genes in a genome, with some penalty for non-maximal segments. The `x` property determines the segment length for this computation. The `calls` property allows you to track how many times a problem instance has been called. The `make_problem(a)` function makes it easy to create problem instances with different segment lengths.

In [177]:
import random
import lab9_lib


def mutate(ind, problem_instance, fitness):
    """mutate one random gene and return mutated part if fitness is better. Full GA could be used here"""
    f1 = fitness(ind)
    if f1 == 1.0:
        return ind, f1

    mutated = ind.copy()
    i = random.randrange(problem_instance)
    mutated[i] = 1 - mutated[i]
    f2 = fitness(mutated)

    if f2 > f1:
        return mutated, f2

    return ind, f1


def split_progenitor(progenitor, genome_length, problem_instance):
    """split progenitors in parts of length problem_instance"""
    end = (
        genome_length
        if genome_length % problem_instance == 0
        else genome_length - problem_instance
    )  # for non-divisible genome_length by problem_instance
    parts = []
    for i in range(0, end, problem_instance):
        parts.append(progenitor[i : i + problem_instance])

    return parts


def run(problem_instance, genome_length):
    """run the algorithm:
    1. create progenitor
    2. split progenitor in parts
    3. mutate parts until fitness is 1.0
    4. join parts in individual
    5. return number of fitness calls and if individual is correct
    """

    fitness = lab9_lib.make_problem(problem_instance)

    progenitor = random.choices([0, 1], k=genome_length)
    parts = split_progenitor(progenitor, genome_length, problem_instance)

    evolved_parts = []
    for part in parts:
        fit = 0
        while fit < 1.0:
            part, fit = mutate(part, problem_instance, fitness)
        evolved_parts.append(g for g in part)

    individual = [gene for part in evolved_parts for gene in part]
    return fitness.calls, sum(individual) == genome_length


# ---------------------------------------------------

GENOME_LENGTH = 1000
instances = [1, 2, 5, 10]
# instances = [1, 2, 3, 5, 7, 10, 20, 50, 100, 200, 500] # for testing

for instance in instances:
    print(f"Instance {instance} - {run(instance, GENOME_LENGTH)}")

Instance 1 - (1487, True)
Instance 2 - (2024, True)
Instance 5 - (3041, True)
Instance 10 - (4598, True)
